## Baseline Model - OPT Implementation

Check if necessary packages are installed and import.

In [1]:
using Pkg
 for p in ("ArgParse", "Knet", "AutoGrad")
    if !haskey(Pkg.installed(),p)
        Pkg.add(p)
    end
 end
    
using ArgParse, Knet, AutoGrad, Statistics

┌ Warning: Pkg.installed() is deprecated
└ @ Pkg /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/Pkg/src/Pkg.jl:570
┌ Warning: Pkg.installed() is deprecated
└ @ Pkg /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/Pkg/src/Pkg.jl:570
┌ Warning: Pkg.installed() is deprecated
└ @ Pkg /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/Pkg/src/Pkg.jl:570


Give the bag of elements as a string and define a empty canvas

In [91]:
B = "H3C4O1";
C = [];

Below is an array of *test_bags* provided in the paper. I will use these for testing purposes as well for the implementation of the OPT agent.

In [96]:
test_bags = ["C3H5NO3", "C4H7N", "C3H8O", "C7H10O2", "C7H8N2O2", "C7H10O2", "C7H8N2O2", "C7H10O2", "C7H8N2O2"]

9-element Vector{String}:
 "C3H5NO3"
 "C4H7N"
 "C3H8O"
 "C7H10O2"
 "C7H8N2O2"
 "C7H10O2"
 "C7H8N2O2"
 "C7H10O2"
 "C7H8N2O2"

*get_bag_from_string* converts a string into an array of elements, taking into account the count for each element

In [128]:
function get_bag_from_string(B)
    
    bag = Char[]
    
    for i = firstindex(B):lastindex(B)
        
        curr_char = B[i]
        
        if !isnumeric(B[i])
            
            push!(bag, curr_char)
            global last_char = curr_char  # Why is global necessary in this case?
            
        else
        
            char_count = parse(Int, B[i])
    
            for _ = 1:char_count - 1
                push!(bag, last_char)
            end
        end
        
    end
    
    return bag
    
end;

Let's try to parse the test bags, and see if there are any issues

In [129]:
for i in test_bags
    println(rpad(i, 12), get_bag_from_string(i))
end

C3H5NO3     ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'N', 'O', 'O', 'O']
C4H7N       ['C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'N']
C3H8O       ['C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'O']
C7H10O2     ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'H', 'O', 'O']
C7H8N2O2    ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'N', 'N', 'O', 'O']
C7H10O2     ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'H', 'O', 'O']
C7H8N2O2    ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'N', 'N', 'O', 'O']
C7H10O2     ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'H', 'O', 'O']
C7H8N2O2    ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'N', 'N', 'O', 'O']


Here **OPT** stands for an **optimal** agent. In this case there is no learning involved, but the atoms are placed on the **Canvas (C)** in a way that is physically optimal chosen from the **Bag (B)**. The implementation of OPT agent works as below (This is directly taken from the appendix of the paper):

1. If the canvas $\mathcal{C}_t$ is not empty, randomly choose a focal atom $f$ from the list of available atoms on the canvas. An atom is considered available if its number of neighbors is less than a predefined number that depends on its element (e.g., one for hydrogen and four for carbon). Two atoms on the canvas are neighbors if their Euclidean distance is below 1.5 $\overset{\circ}{\textbf{A}}$. If there are no available atoms on the canvas, a focal atom is randomly chosen from the list of atoms on the canvas.

2. Randomly choose and element $e_t$ from the bag $\mathcal{B}_t$.

3. Randomly place the atom $a_t = (e_t, x_t)$ on a sphere with a radial distance $d = 1.1 \overset{\circ}{\textbf{A}}$ around $x_f$ to obtain $\mathcal{C}_{t+1,\text{raw}}$. If the canvas is empty, place the atom at the origin.

4. Optimize only the position of $a_t$ using $F$ to obtain $\mathcal{C}_{t+1,\text{opt}}$.

5. Compute the energy difference $\Delta E( \mathcal{C}_{t+1,\text{opt}} ) - [E( \mathcal{C}_t ) + E({e_t, 0})]$.

6. If $\Delta E(t) > 0$, return $e_t$ to the bag and go back to step 1.

7. Optimize canvas $\mathcal{C}_{t+1,\text{opt}}$ using $F$ to obtain $\mathcal{C}_{t+1}$.

8. Increment $t$ by 1.

9. If the bag is not empty, go back to step 1.
